# Check in on Swarm

### Import Python tools and Jupyter configuration

In [6]:
%load_ext lab_black

In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import sqlite3
from datetime import datetime

/Users/mhustiles/.local/share/virtualenvs/notebooks-xu8v5Vvv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [8]:
import altair as alt
import altair_latimes as lat
import keys

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

In [9]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Find a venue

In [10]:
today = datetime.today().strftime("%Y%m%d")

In [11]:
# DEL REY: 4d27f64f467d6ea88416c295

In [12]:
import requests
import json

oauth_token = keys.TOKEN

query = "Del Rey"

# Location lat,lon
near = "Los Angeles, CA"

# Today's date for version
v = today

url = "https://api.foursquare.com/v2/venues/search?oauth_token={}&query={}&v={}&near={}"

In [13]:
r = requests.post(url.format(oauth_token, query, v, near))

In [14]:
r.json()

{'meta': {'code': 200, 'requestId': '6147e9740580187d047094b6'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4be71411bcef2d7f4ad405e5',
    'name': 'Marina del Rey Harbor',
    'contact': {'phone': '4243544200', 'formattedPhone': '(424) 354-4200'},
    'location': {'address': '4640 Admiralty Way',
     'lat': 33.9816425,
     'lng': -118.4409761,
     'labeledLatLngs': [{'label': 'display',
       'lat': 33.9816425,
       'lng': -118.4409761}],
     'postalCode': '90292',
     'cc': 'US',
     'city': 'Marina del Rey',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['4640 Admiralty Way', 'Marina del Rey, CA 90292']},
    'categories': [{'id': '4bf58dd8d48988d1e0941735',
      'name': 'Harbor / Marina',
      'pluralName': 'Harbors / Marinas',
      'shortName': 'Harbor / Marina',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/harbor_',
       'suffix': '.p